In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import sys
import os

sys.path.append('../tools')
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import sklearn
import random

import torchvision.transforms as T
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks
from models.rns_dataloader import get_data, get_data_by_episode

import data_utility
import annotation_utility
import interactive_plot

import warnings

warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")
warnings.filterwarnings("ignore", ".*Set a lower value for log_every_n_steps if you want to see logs for the training epoch*")
warnings.filterwarnings("ignore", ".*exists and is not empty*")
warnings.filterwarnings("ignore", ".*Checkpoint directory {dirpath} exists and is not empty*")


In [3]:
data_dir = "../../../user_data/"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'

random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    # True ensures the algorithm selected by CUFA is deterministic
    torch.backends.cudnn.deterministic = True
    # torch.set_deterministic(True)
    # False ensures CUDA select the same algorithm each time the application is run
    torch.backends.cudnn.benchmark = False

import pytorch_lightning

pytorch_lightning.utilities.seed.seed_everything(seed=random_seed, workers=True)

Global seed set to 42


42

In [4]:
from models.rns_dataloader import RNS_Downstream
# from models.SwaV import SwaV

In [5]:
import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset, SwaVCollateFunction
from lightly.loss import SwaVLoss
from lightly.loss.memory_bank import MemoryBankModule
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes


In [6]:
def collate_fn(batch):
    info = list(zip(*batch))
    data = info[0]
    label = info[1]
    return torch.stack(data), torch.stack(label)

In [7]:
data_list = os.listdir(data_dir+'rns_test_cache')[1:]

# data_list = ['HUP182.npy',   'HUP129.npy',   'HUP109.npy', 'HUP156.npy', 'HUP096.npy', 'RNS026.npy',  'HUP159.npy']
# data_list = ['RNS026.npy', 'HUP159.npy', 'HUP129.npy', 'HUP096.npy', 'HUP182.npy']
train_data, train_label, test_data, test_label, train_index, test_index = get_data(data_list, split=0.8)
# data, label,_,_ = get_data(data_list, split=1)
# train_data, test_data, train_label, test_label = sklearn.model_selection.train_test_split(data, label, test_size=0.8, random_state=42)

print(train_data.shape)
print(train_label.shape)
print(train_index.shape)
print(test_data.shape)
print(test_label.shape)
print(test_index.shape)

100%|██████████| 16/16 [00:26<00:00,  1.66s/it]


(87319, 249, 36)
(87319,)
(87319,)
(21837, 249, 36)
(21837,)
(21837,)


In [8]:
import torch.nn.functional as F
from torch import nn
import torch
import sklearn
from sigmoid_loss import sigmoid_focal_loss

class LinearHead(pl.LightningModule):
    def __init__(self, backbone,):
        super().__init__()
        self.backbone = backbone
        self.fc1 = nn.Linear(2048, 2)
        self.softmax = nn.Softmax(dim=1)
        self.alpha = 0
        self.gamma = 5

    def training_step(self, batch, batch_idx):
        x, y = batch
        self.set_requires_grad(self.backbone, False)
        x = self.backbone(x)
        x = x.view(-1, 2048)
        pred = self.fc1(x)
        pred = self.softmax(pred)
        label = F.one_hot(y, num_classes=2).squeeze()
        loss = sigmoid_focal_loss(pred.float(), label.float(), alpha=self.alpha, gamma=self.gamma, reduction='mean')
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = self.backbone(x)
        x = x.view(-1, 2048)
        pred = self.fc1(x)
        pred = self.softmax(pred)
        label = F.one_hot(y, num_classes=2).squeeze()
        loss = sigmoid_focal_loss(pred.float(), label.float(), alpha=self.alpha, gamma=self.gamma, reduction='mean')
        out = torch.argmax(pred, dim=1)
        # print(out.size)
        out = out.detach().cpu().numpy()
        target = y.squeeze().detach().cpu().numpy()
        fscore = sklearn.metrics.f1_score(out, target,labels = [0,1],zero_division=0)
        acc = sklearn.metrics.accuracy_score(out, target)
        # print(acc)
        # print(precision)
        # print(recall)
        # print(fscore)
        # Logging to TensorBoard (if installed) by default
        self.log("val_loss", loss,prog_bar=False)
        self.log("val_acc", acc,prog_bar=False)
        self.log("val_fscore", fscore,prog_bar=False)
        return pred, label

    def predict_step(self, batch, batch_idx):
        # print(batch)
        x, y = batch
        emb = self.backbone(x)
        emb = emb.view(-1, 2048)
        pred = self.fc1(emb)
        # Logging to TensorBoard (if installed) by default
        return pred, y, emb

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def set_requires_grad(self, model, requires_grad=True, exclude = None):
        """
        Used in training adversarial approach
        :param model:
        :param requires_grad:
        :return:
        """
        for param in model.parameters():
            param.requires_grad = requires_grad

        if exclude is not None:
            for name, child in model.named_children():
                if name in exclude:
                    for param in child.parameters():
                        param.requires_grad =not requires_grad

In [9]:
class SwaV(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SwaVProjectionHead(2048, 2048, 128)
        self.prototypes = SwaVPrototypes(128, 2048, 1)

        self.start_queue_at_epoch = 35
        self.queues = nn.ModuleList([MemoryBankModule(size=256) for _ in range(2)])

    def forward(self, high_resolution, low_resolution, epoch):
        self.prototypes.normalize()

        high_resolution_features = [self._subforward(x) for x in high_resolution]
        low_resolution_features = [self._subforward(x) for x in low_resolution]

        high_resolution_prototypes = [
            self.prototypes(x, epoch) for x in high_resolution_features
        ]
        low_resolution_prototypes = [
            self.prototypes(x, epoch) for x in low_resolution_features
        ]
        queue_prototypes = self._get_queue_prototypes(high_resolution_features, epoch)

        return high_resolution_prototypes, low_resolution_prototypes, queue_prototypes

    def _subforward(self, input):
        features = self.backbone(input).flatten(start_dim=1)
        features = self.projection_head(features)
        features = nn.functional.normalize(features, dim=1, p=2)
        return features

    @torch.no_grad()
    def _get_queue_prototypes(self, high_resolution_features, epoch):
        if len(high_resolution_features) != len(self.queues):
            raise ValueError(
                f"The number of queues ({len(self.queues)}) should be equal to the number of high "
                f"resolution inputs ({len(high_resolution_features)}). Set `n_queues` accordingly."
            )

        # Get the queue features
        queue_features = []
        for i in range(len(self.queues)):
            _, features = self.queues[i](high_resolution_features[i], update=True)
            # Queue features are in (num_ftrs X queue_length) shape, while the high res
            # features are in (batch_size X num_ftrs). Swap the axes for interoperability.
            features = torch.permute(features, (1, 0))
            queue_features.append(features)

        # If loss calculation with queue prototypes starts at a later epoch,
        # just queue the features and return None instead of queue prototypes.
        if self.start_queue_at_epoch > 0 and epoch < self.start_queue_at_epoch:
            return None

        # Assign prototypes
        queue_prototypes = [self.prototypes(x, epoch) for x in queue_features]
        return queue_prototypes

In [10]:
linear_eval = 'p=0'

ckpt = torch.load(ckpt_folder_root+"checkpoint99.pth")
resnet = torchvision.models.resnet50()

backbone = nn.Sequential(*list(resnet.children())[:-1])
swav = SwaV(backbone)
swav.load_state_dict(ckpt['model_state_dict'])

model = LinearHead(swav.backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
ckpt_save_n_step = 150

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='train_loss',
                                                   filename= linear_eval + '-{step}-{train_loss:.5f}',
                                                   dirpath=ckpt_folder_root + 'rns_linear_eval/' + linear_eval + '/',
                                                   save_top_k=-1,
                                                   every_n_train_steps=ckpt_save_n_step,
                                                   save_on_train_epoch_end=False)

early_stop_callback = pl_callbacks.EarlyStopping(monitor="val_fscore",
                                                 patience=10,
                                                 verbose=False,
                                                 mode="max")

csv_logger = pl_loggers.CSVLogger(ckpt_folder_root + 'rns_linear_eval/' + linear_eval + '/',
                                  name='logger')

trainer = pl.Trainer(logger=csv_logger,
                     max_epochs=100,
                     callbacks=[checkpoint_callback, early_stop_callback],
                     # callbacks=[checkpoint_callback],
                     accelerator='gpu',
                     devices=1,
                     log_every_n_steps=50,
                     precision=16,
                     check_val_every_n_epoch=None,
                     val_check_interval=ckpt_save_n_step,
                     enable_model_summary=False,
                     )

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
from models.rns_dataloader import RNS_Downstream
train_dataset = RNS_Downstream(train_data, train_label, transform=True, astensor=True)
test_dataset = RNS_Downstream(test_data, test_label, transform=False, astensor=True)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)

val_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)

trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data loaded
(87319, 249, 36)
(87319,)
data loaded
(21837, 249, 36)
(21837,)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
test_dataset = RNS_Downstream(test_data, test_label, transform=False, astensor=True)
val_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)

In [12]:
predictions = trainer.predict(model,val_dataloader,ckpt_path=ckpt_folder_root+ 'rns_linear_eval/p=0/p=0-step=150-train_loss=0.02511.ckpt')

Restoring states from the checkpoint path at ../../../user_data/checkpoints/rns_linear_eval/p=0/p=0-step=150-train_loss=0.02511.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../../user_data/checkpoints/rns_linear_eval/p=0/p=0-step=150-train_loss=0.02511.ckpt


Predicting: 286it [00:00, ?it/s]

In [13]:
output_list = []
target_list = []
emb_list = []
m = nn.Softmax(dim=1)
for pred, y, emb in predictions:
    output_list.append(pred)
    target_list.append(y)
    emb_list.append(emb)

In [14]:
pred_raw = torch.vstack(output_list)
target = torch.vstack(target_list)
emb = torch.vstack(emb_list)
out = torch.argmax(pred_raw, dim=1)

In [15]:
sklearn.metrics.accuracy_score(torch.argmax(pred_raw, dim=1), target)

0.8566652928515822

In [16]:
clf_report = sklearn.metrics.classification_report(torch.argmax(pred_raw, dim=1), target, digits=6)

print(f"Classification Report : \n{clf_report}")

Classification Report : 
              precision    recall  f1-score   support

           0   0.924500  0.872484  0.897739     15747
           1   0.712156  0.815764  0.760447      6090

    accuracy                       0.856665     21837
   macro avg   0.818328  0.844124  0.829093     21837
weighted avg   0.865281  0.856665  0.859450     21837



In [17]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(target, m(pred_raw.float())[:,1], pos_label=1)
sklearn.metrics.auc(fpr, tpr)

0.8812383708296269

In [18]:
split_ind = np.insert(np.where(np.diff(test_index['episode_index'])!=0)[0],0,-1)
split_ind = np.insert(split_ind, split_ind.size, len(test_index))

In [19]:
len(test_index)

21837

In [20]:
split_ind[-1]

21837

In [21]:
test_label_split = []
pred_label_split = []
for i in range(len(split_ind)-1):
    start_ind = split_ind[i]
    end_ind = split_ind[i+1]
    test_label_split.append(test_label[start_ind+1:end_ind+1])
    pred_label_split.append(out[start_ind+1:end_ind+1])


In [22]:
pred_label_split[0]

tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
test_label_split[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [24]:
sklearn.metrics.accuracy_score([np.sign(tl.sum()) for tl in test_label_split], [np.sign(tl.sum()) for tl in pred_label_split])

0.7709923664122137

In [25]:
clf_report = sklearn.metrics.classification_report([np.sign(tl.sum()) for tl in test_label_split], [np.sign(tl.sum()) for tl in pred_label_split], digits=4)

print(f"Classification Report : \n{clf_report}")

Classification Report : 
              precision    recall  f1-score   support

         0.0     1.0000    0.4915    0.6591       118
         1.0     0.7059    1.0000    0.8276       144

    accuracy                         0.7710       262
   macro avg     0.8529    0.7458    0.7433       262
weighted avg     0.8383    0.7710    0.7517       262



In [ ]:
test_index